
FINAL PIPELINE


In [1]:
import pandas as pd

# File path to the dataset
file_path = r'C:\Users\asusa\Downloads\N-Beats Final Model (lb = 1200)\Aggregated_Data.csv'

# Chunk size
chunk_size = 100000

# Function to check for date inconsistencies and filter by date range
def filter_and_check_data(df):
    """
    Filters the data by date range and checks for date inconsistencies.

    Args:
      df: A pandas DataFrame containing the data.

    Returns:
      A tuple containing:
        - A DataFrame filtered by date range (2020-2023) and station (Sta == 211).
        - A boolean indicating if any dates are out of order.
        - A list of indices where dates are out of order.
        - A list of tuples, where each tuple represents a missing date range.
    """

    # Keep only relevant columns
    relevant_columns = ['_id', 'Dt', 'Sta', 'St', 'Line', 'EqN', 'Tag']
    if not set(relevant_columns).issubset(df.columns):
        raise ValueError(f"Missing required columns: {set(relevant_columns) - set(df.columns)}")

    df = df[relevant_columns]

    # # Filter to include only Sta == 211
    # df = df[df['Sta'] == 211]

    # Convert 'Dt' to datetime format
    df['Dt'] = pd.to_datetime(df['Dt'], errors='coerce', utc=True)

    # Remove rows where 'Dt' couldn't be converted to datetime
    df = df.dropna(subset=['Dt'])

    # Filter data to include only dates between 2020 and 2023
    df = df[(df['Dt'] >= '2020-01-01') & (df['Dt'] <= '2023-12-31')]

    dates_out_of_order = False
    out_of_order_indices = []
    missing_dates = []

    # Check if there are valid dates in the chunk
    if len(df) == 0:
        return df, dates_out_of_order, out_of_order_indices, missing_dates

    # Check for ascending order
    if not df['Dt'].is_monotonic_increasing:
        dates_out_of_order = True
        for i in range(1, len(df)):
            if df['Dt'].iloc[i] < df['Dt'].iloc[i - 1]:
                out_of_order_indices.append(i)

    # Check for gaps (missing dates)
    daily_dates = df['Dt'].dt.date.unique()
    
    # Ensure there are valid dates before checking for missing dates
    if len(daily_dates) > 0:
        full_date_range = pd.date_range(start=daily_dates.min(), end=daily_dates.max(), freq='D')
        missing_dates = [(str(d1.date()), str(d2.date())) for d1, d2 in zip(full_date_range, full_date_range[1:]) if d2.date() not in daily_dates]

    return df, dates_out_of_order, out_of_order_indices, missing_dates

# Initialize lists to store results across chunks
filtered_data_chunks = []
out_of_order_all = False
out_of_order_indices_all = []
missing_dates_all = []

# Read the dataset in chunks
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    # Filter data and check for inconsistencies in each chunk
    filtered_df, dates_out_of_order, out_of_order_indices, missing_dates = filter_and_check_data(chunk)

    # Append filtered chunk
    filtered_data_chunks.append(filtered_df)

    # Collect out-of-order indices and missing dates if any
    if dates_out_of_order:
        out_of_order_all = True
        out_of_order_indices_all.extend(out_of_order_indices)
    
    if missing_dates:
        missing_dates_all.extend(missing_dates)

# Concatenate all filtered chunks into a single DataFrame

filtered_df = pd.concat(filtered_data_chunks)

# Print results
if out_of_order_all:
    print("Dates out of order found!")
    # Print the first 10 rows with out-of-order dates
    print(filtered_df.iloc[out_of_order_indices_all[:10]].to_markdown(index=False, numalign="left", stralign="left"))

if missing_dates_all:
    print("Missing dates found!")
    # Print the first few missing date ranges
    print("First 10 missing date ranges:", missing_dates_all[:10])

if not out_of_order_all and not missing_dates_all:
    print("No date inconsistencies found!")

# Optionally save the filtered data

filtered_df.to_csv(r'filtered_2020_2023_Sta_211.csv', index=False)


C:\Users\asusa\AppData\Local\Temp\ipykernel_55872\1254148076.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Dt'] = pd.to_datetime(df['Dt'], errors='coerce', utc=True)
C:\Users\asusa\AppData\Local\Temp\ipykernel_55872\1254148076.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Dt'] = pd.to_datetime(df['Dt'], errors='coerce', utc=True)
C:\Users\asusa\AppData\Local\Temp\ipykernel_55872\1254148076.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

Dates out of order found!
| _id                      | Dt                        | Sta   | St   | Line   | EqN    | Tag   |
|:-------------------------|:--------------------------|:------|:-----|:-------|:-------|:------|
| 63bbd6bd1f70e5b6d8fde690 | 2020-01-03 16:54:12+00:00 | 207   | 0    | 7      | 151776 | 3031  |
Missing dates found!
First 10 missing date ranges: [('2020-03-23', '2020-03-24'), ('2020-03-24', '2020-03-25'), ('2020-03-25', '2020-03-26'), ('2020-03-26', '2020-03-27'), ('2020-03-27', '2020-03-28'), ('2020-03-28', '2020-03-29'), ('2020-03-29', '2020-03-30'), ('2020-03-30', '2020-03-31'), ('2020-03-31', '2020-04-01'), ('2020-04-01', '2020-04-02')]


In [2]:
!pip install tabulate